# Formatting stuff

In [257]:
#housekeeping and essential imports
import pandas as pd
import os

pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1800)
pd.set_option ('display.max_colwidth', 150)
pd.set_option('display.max_columns', 50)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [258]:
# surpress warnings
import warnings
warnings.filterwarnings('ignore')

In [361]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [360]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')
#plt.style.use('seaborn-dark-palette')
#plt.style.use('seaborn')

---

# Pipeline summary
0. **ACTION ITEM**: normalize bams. make sure all samples must have the same amountof reads/alignements, but within 2 replicates, they can vary.
1. Get *annotation:*
    - make `BED` of protein coding genes (__`/gpfs/commons/groups/sanjana_lab/cdai/TFscreen/atac/Protein_coding_genes.bed`__):
    - makde `BED` of promoter region of protein coding genes(__`/gpfs/commons/groups/sanjana_lab/cdai/TFscreen/atac/Protein_coding_gene_promoters.bed`__):
        - expand TSS up 2000 down 500: `slobBed -i input -g genomesize -l 2000 -r 500 -s` -s for stranded expansion. I ended up using python script instead
2. annotate each peak (__`/gpfs/commons/groups/sanjana_lab/cdai/TFscreen/atac/macs2/v6/*.broadPeak`__) with both promoter region and gene body (bash script __`annotate_peaks.sh`__)
    
3. Get peak counts and read counts per feature (gene body or gene promoter region). python script below. 

4. Use R scripts to do downstream analysis.

---

## Python Script

### get TSS and gene body annotation from gencode GTF

In [12]:
gtf = pd.read_csv('/c/groups/sanjana_lab/cdai/ref_genome/gencode.v31.primary_assembly.annotation.pandas.df.txt', sep='\t')

In [19]:
gtfColNames=['seqname', 'start', 'end', 'gene_id', 'gene_name' , 'strand']

In [112]:
# protein coding genes only
protein_coding_genes = gtf[(gtf.feature == 'gene') & (gtf.gene_type.isin(['protein_coding']) & 
                           (gtf.seqname.str.len() <6)) & (gtf.seqname != 'chrM')][gtfColNames]

In [ ]:
# get promoter regions
promoters = gtf[(gtf.feature == 'gene') & (gtf.gene_type.isin(['protein_coding']) & 
                           (gtf.seqname.str.len() <6)) & (gtf.seqname != 'chrM')][gtfColNames] # first get list of protein coding genes
promoters['TSS'] = promoters.apply(lambda x: x.start if x.strand == '+' else x.end, axis=1) # then get exact Transcription START coordinate

# get range of promoter region based on the gene's strandedness
promoters['start'] = promoters.apply(lambda x: x.TSS - 2000 if x.strand == '+' else x.TSS + 2000, axis=1)
promoters['end'] = promoters.apply(lambda x: x.TSS + 500 if x.strand == '+' else x.TSS - 500, axis=1)

# correct [start, end] coordinates, smaller value on the left (otherwise, pedtools intersect won't give any result)
start = [min(x[0],x[1]) for x in list(zip(promoters['start'], promoters['end']))]
end = [max(x[0],x[1]) for x in list(zip(promoters['start'], promoters['end']))]
promoters['start'] = start
promoters['end'] = end

promoters = promoters[gtfColNames]

In [121]:
promoters.to_csv('Protein_coding_gene_promoters.bed', index=False, header=None, sep='\t')
protein_coding_genes.to_csv('Protein_coding_genes.bed', index=False, header=None, sep='\t')

---

## BASH script

`/c/groups/sanjana_lab/cdai/TFscreen/atac/macs2/v6/annotate_peaks.sh`

---

## Python Script

### Generate readcount / peakcount matrix:

In [126]:
import pandas as pd
import os
import glob

In [125]:
os.chdir('/c/groups/sanjana_lab/cdai/TFscreen/atac/macs2/v6')

In [242]:
gtf = pd.read_csv('/c/groups/sanjana_lab/cdai/ref_genome/gencode.v31.primary_assembly.annotation.pandas.df.txt', sep='\t')
gencode_gene_list = gtf[gtf.feature == 'gene'].gene_id # get gencode gene only gene-id

In [335]:
def getAtacCountMatrix(filenames, samplenames, Nsamples, gene_list):
    """Provide a list of filenames, and extracted sample names. Return 2 dataframes:
    1. 60622 rows of gene_id, n columns of samples, each column is read counts of all peaks in the gene region or promoter region
    2. 60622 rows of gene_id, n columns of samples, each column is number of peaks of the gene region or promoter region"""
    
    readcounts_df = pd.DataFrame(gene_list)
    peakcounts_df = pd.DataFrame(gene_list)

    for f,a in zip(filenames,samplenames):
        quantReads = pd.read_csv(f,sep='\t', header=None) # read peak calling file
        quantReads.rename(columns={3:'peak_name', 9:'readcount' , 13:'gene_id', 14:'gene_name'}, inplace=True) # rename a few columns to make it more readable
        reads_per_gene = quantReads.groupby(by=['gene_id'])['readcount'].sum().reset_index().rename(columns={'readcount':a}) # sum reads per gene, name column by sample name
        peaks_per_gene = quantReads.groupby(by=['gene_id'])['peak_name'].size().reset_index().rename(columns={'peak_name':a}) # sum number of peaks per gene
        readcounts_df = pd.merge(readcounts_df, reads_per_gene, how='left', on='gene_id', ) # add column of sample data
        peakcounts_df = pd.merge(peakcounts_df, peaks_per_gene, how='left', on='gene_id') # add column of sample data

    column_order = ['gene_id'] + ['A'+str(x) for x in range(1,Nsamples+1)]
    readcounts_df = readcounts_df[column_order]
    peakcounts_df = peakcounts_df[column_order]
    
    return readcounts_df, peakcounts_df

##### get ATAC-seq promoter region readcounts and peak counts

In [334]:
filenames = glob.glob('*_x_promoter.bed') # get filenames of ATACseq 
samplenames = ['A'+ x.split('.')[0].split('_')[0].strip('ATAC') for x in filenames] # extract sample names from file names

In [336]:
readcounts_df, peakcounts_df = getAtacCountMatrix(filenames, samplenames, 12, gencode_gene_list)

In [332]:
readcounts_df.to_csv('ATAC_Promoter_Peak_rawreadcounts.txt', sep='\t', header=True, index=False)
peakcounts_df.to_csv('ATAC_Promoter_NumberofPeaks.txt', sep='\t', header=True, index=False)

##### get ATAC-seq gene body region (gene code feature=gene coordinates) readcounts and peak counts

In [339]:
fn = glob.glob('*_x_gene.bed')

In [341]:
sn = ['A'+ x.split('.')[0].split('_')[0].strip('ATAC') for x in fn] # extract sample names from file names

In [345]:
readcounts_df2, peakcounts_df2 = getAtacCountMatrix(fn, sn, 12, gencode_gene_list)

In [350]:
readcounts_df2.to_csv('ATAC_genebody_Peak_rawreadcounts.txt', sep='\t', header=True, index=False)
peakcounts_df2.to_csv('ATAC_genebody_NumberofPeaks.txt', sep='\t', header=True, index=False)

---